In [ ]:
!wget https://raw.githubusercontent.com/adobe/SimpleSensor/master/simplesensor/collection_modules/demographic_camera/classifiers/haarcascades/haarcascade_mcs_nose.xml
!wget https://raw.githubusercontent.com/adobe/SimpleSensor/master/simplesensor/collection_modules/demographic_camera/classifiers/haarcascades/haarcascade_smile.xml

In [1]:
import cv2

def overlay_transparent(background_img, img_to_overlay_t, x, y, overlay_size=None):
    bg_img = background_img.copy()
    if bg_img.shape[2] == 3:
        bg_img = cv2.cvtColor(bg_img, cv2.COLOR_BGR2BGRA)

    if overlay_size is not None:
        img_to_overlay_t = cv2.resize(img_to_overlay_t.copy(), overlay_size)

    b, g, r, a = cv2.split(img_to_overlay_t)

    mask = cv2.medianBlur(a, 5)

    h, w, _ = img_to_overlay_t.shape
    roi = bg_img[y:y + h, x:x + w]

    if roi.shape[0] == h and roi.shape[1] == w:
        img1_bg = cv2.bitwise_and(roi.copy(), roi.copy(), mask=cv2.bitwise_not(mask))
        img2_fg = cv2.bitwise_and(img_to_overlay_t, img_to_overlay_t, mask=mask)

        bg_img[y:y + h, x:x + w] = cv2.add(img1_bg, img2_fg)
        bg_img = cv2.cvtColor(bg_img, cv2.COLOR_BGRA2BGR)

    return bg_img

cap = cv2.VideoCapture(0)
prev_nose_x, prev_nose_y = 0, 0
prev_mouth_x, prev_mouth_y = 0, 0

face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
nose_cascade = cv2.CascadeClassifier('haarcascade_mcs_nose.xml')
mouth_cascade = cv2.CascadeClassifier('haarcascade_smile.xml')

ears_img = cv2.imread('snapchatFilter/ears.png', -1)
nose_img = cv2.imread('snapchatFilter/nose.png', -1)
mouth_img = cv2.imread('snapchatFilter/mouth.png', -1)

while True:
    ret, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    for (x, y, w, h) in faces:
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = frame[y:y+h, x:x+w]

        noses = nose_cascade.detectMultiScale(roi_gray, 1.3, 5)
        for (nx, ny, nw, nh) in noses:
            frame = overlay_transparent(frame, nose_img, x + nx, y + ny, (nw, nh))

    
        mouths = mouth_cascade.detectMultiScale(roi_gray, 1.8, 20)
        for (mx, my, mw, mh) in mouths:
            overlay_width = mw 
            overlay_height = mh
            overlay_x = x + mx + mw // 2 - overlay_width // 2 
            overlay_y = y + my + mh -25 

            frame = overlay_transparent(frame, mouth_img, overlay_x, overlay_y, (overlay_width, overlay_height))


        # overlay ears
        frame = overlay_transparent(frame, ears_img, x, y - h//5, (w, h//3))

    cv2.imshow('frame', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


AttributeError: 'NoneType' object has no attribute 'copy'